## Tokenizing .loom or .h5ad single cell RNA-seq data to rank value encoding .dataset format

#### Input data is a directory with .loom or .h5ad files containing raw counts from single cell RNAseq data, including all genes detected in the transcriptome without feature selection. The input file type is specified by the argument file_format in the tokenize_data function.

#### The discussion below references the .loom file format, but the analagous labels are required for .h5ad files, just that they will be column instead of row attributes and vice versa due to the transposed format of the two file types.

#### Genes should be labeled with Ensembl IDs (loom row attribute "ensembl_id"), which provide a unique identifer for conversion to tokens. Other forms of gene annotations (e.g. gene names) can be converted to Ensembl IDs via Ensembl Biomart. Cells should be labeled with the total read count in the cell (loom column attribute "n_counts") to be used for normalization.

#### No cell metadata is required, but custom cell attributes may be passed onto the tokenized dataset by providing a dictionary of custom attributes to be added, which is formatted as loom_col_attr_name : desired_dataset_col_attr_name. For example, if the original .loom dataset has column attributes "cell_type" and "organ_major" and one would like to retain these attributes as labels in the tokenized dataset with the new names "cell_type" and "organ", respectively, the following custom attribute dictionary should be provided: {"cell_type": "cell_type", "organ_major": "organ"}. 

#### Additionally, if the original .loom file contains a cell column attribute called "filter_pass", this column will be used as a binary indicator of whether to include these cells in the tokenized data. All cells with "1" in this attribute will be tokenized, whereas the others will be excluded. One may use this column to indicate QC filtering or other criteria for selection for inclusion in the final tokenized dataset.

#### If one's data is in other formats besides .loom or .h5ad, one can use the relevant tools (such as Anndata tools) to convert the file to a .loom or .h5ad format prior to running the transcriptome tokenizer.

**********************************************************************************************************
#### OF NOTE: PLEASE ENSURE THE CORRECT TOKEN DICTIONARY AND GENE MEDIAN FILE IS USED FOR THE CORRECT MODEL.
#### 95M: current defaults; 30M: https://huggingface.co/ctheodoris/Geneformer/tree/main/geneformer/gene_dictionaries_30m

#### ADDITIONALLY:
#### The 95M model series require the special_token argument to be set to True and model_input_size to be 4096. (current defaults)
#### The 30M model series require the special_token argument to be set to False and the model_input_size to be 2048.

In [1]:
from geneformer import TranscriptomeTokenizer

/data/mcri_heartv2/mcri_nedar/env/conda/geneformerV2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-28 06:55:55.636260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745794555.654287  971908 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745794555.659881  971908 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 06:55:55.678679: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions

In [2]:
#copy a subset of .loom files to another folder to generate resr.dataset.

import os
import shutil

# Define the list of strings to match against
test = ["1290_2", "1290_1", "1606_1", "1610_1", "1610_2", "1622_2", "1603_2", 
        "TWCM-10-5", "TWCM-13-36", "TWCM-11-3", "TWCM-13-96", "TWCM-13-80", 
        "TWCM-13-168", "TWCM-11-192", "BO_H28_LV0", "HCAHeart7880860", "HCAHeart7829978", 
        "IC_H02_LV0", "BO_H35_LV0", "BO_H36_LV0", "H0020_LV", "IC_H01_LV0", 
        "Young3_GSM5606415", "Adult3_GSM4742856", "Young2_GSM5606414"]

# Specify the source and destination directories
source_dir = '/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/loom_file_path_CM'  # Replace with the path to your source folder
dest_dir = '/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom'  # Replace with the path to your destination folder

# Loop through each file in the source directory
for filename in os.listdir(source_dir):
    if filename.endswith('.loom'):
        # Check if any of the strings in `test` is present in the file name
        if any(substring in filename for substring in test):
            # If a match is found, copy the file to the destination folder
            shutil.copy(os.path.join(source_dir, filename), os.path.join(dest_dir, filename))


In [3]:
#in vitro     #in vitro siRNA add these as well --> "treatment":"treatment", "orig.ident":"orig.ident", "condition":"condition"
tk = TranscriptomeTokenizer({"cell_type": "cell_type", "replicate": "replicate", "unique_cell_id":"unique_cell_id","individual":"individual", "n_counts":"n_counts", "barcodes":"barcodes", "disease": "disease"}, nproc=1)

tk.tokenize_data("/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/", 
                 "/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_dataset", 
                 "20250304_invitro_mutant_DSPKI_TPM1_TTNhet_TNNT2_4096_w_length", 
                 file_format="h5ad")

Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/WT_DSP.h5ad


100%|██████████| 7/7 [00:00<00:00, 12.34it/s]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/WT_DSP.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/WT_TNNT2.h5ad


100%|██████████| 72/72 [00:07<00:00,  9.69it/s]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/WT_TNNT2.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/WT_TPM1.h5ad


...

/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/patient_TNNT2.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/patient_TPM1.h5ad


100%|██████████| 9/9 [00:00<00:00, 14.67it/s]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/patient_TPM1.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/patient_TTNhet.h5ad


100%|██████████| 5/5 [00:00<00:00, 26.83it/s]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/data/mcri_heartv2/mcri_nedar/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/mutant_h5ad/patient_TTNhet.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


In [3]:
#in vivo
tk = TranscriptomeTokenizer({"cell_type": "cell_type", "individual":"individual", "group":"group", "n_counts":"n_counts", "sex":"sex", "age":"age","barcodes":"barcodes", "disease":"disease", "genotype": "genotype"}, nproc=1)
tk.tokenize_data("/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom", 
                 "/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/output_dir_dataset", 
                 "20250412_25samples_human_dcm_control_pediatric_adult_TEST_CMonly_4096_w_length", 
                 file_format="loom")

Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Lavine_TWCM-10-5_DCM.loom


100%|██████████| 1/1 [00:00<00:00,  3.49it/s]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Lavine_TWCM-10-5_DCM__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Lavine_TWCM-11-192_Control.loom


100%|██████████| 3/3 [00:00<00:00,  3.11it/s]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Lavine_TWCM-11-192_Control__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Lavine_TWCM-11-3_DCM.loom


...

/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Seidman_HCAHeart7880860_Adult_Control__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Seidman_IC_H01_LV0_Adult_DCM.loom


100%|██████████| 3/3 [00:01<00:00,  2.98it/s]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Seidman_IC_H01_LV0_Adult_DCM__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Tokenizing /data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Seidman_IC_H02_LV0_Young_DCM.loom


100%|██████████| 4/4 [00:01<00:00,  2.79it/s]


/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/CM_test_loom/Seidman_IC_H02_LV0_Young_DCM__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


In [4]:
from datasets import Dataset

# Load the dataset from a local .dataset file (replace 'your_file.dataset' with your actual file path)
dataset = Dataset.load_from_disk("/data/mcri_heartv2/mcri_nedar/DCM-Foundation-Model-data/output_dir_dataset/20250412_25samples_human_dcm_control_pediatric_adult_TEST_CMonly_4096_w_length.dataset")

# Show the features (columns) in the dataset
print(dataset.column_names)
first_5_rows = dataset[:5]

# Print the first 5 rows
print(first_5_rows)

['input_ids', 'cell_type', 'individual', 'group', 'n_counts', 'sex', 'age', 'barcodes', 'disease', 'genotype', 'length']
{'input_ids': [[2, 9740, 11139, 9637, 4427, 63, 10742, 10608, 1386, 4583, 16263, 8856, 4742, 16797, 1945, 14011, 8687, 8993, 5145, 17006, 1359, 8251, 6025, 13992, 16453, 10046, 7852, 14819, 17364, 14779, 2537, 1436, 2068, 1200, 13074, 529, 9192, 1368, 12649, 7122, 6928, 2843, 12614, 1406, 13510, 14739, 5214, 10145, 12628, 2027, 9617, 8618, 5203, 16341, 7515, 10170, 519, 18981, 10811, 1146, 9783, 10376, 8570, 16518, 14891, 9315, 12871, 256, 4233, 3223, 9225, 17320, 15107, 8628, 770, 9792, 4953, 8919, 7985, 11992, 10906, 8398, 353, 9757, 12335, 5979, 16769, 13036, 111, 3267, 15391, 2728, 8472, 17675, 19073, 9348, 13020, 4844, 6479, 13397, 5226, 3674, 11540, 1276, 6856, 2938, 14454, 9544, 4789, 3583, 18471, 15575, 5020, 1847, 13280, 1633, 488, 3763, 1033, 17072, 9394, 8530, 16754, 9889, 9808, 13514, 9506, 10843, 1988, 9460, 402, 2331, 3633, 6349, 15212, 7651, 3676, 1738